In [1]:
# ЧЕРНОВОЙ ВАРИАНТ импортирует веса из таблиц тестирования
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
with open("config.txt", "w") as fw:
    fw.write("[COLAB]\n")
    fw.write("knowledge_dir=/content/drive/MyDrive/KIA_AVATAR/Scripts/knowledge\n")
    fw.write("prev_knowledge_dir=/content/drive/MyDrive/KIA_AVATAR/Scripts/../knowledge\n")
    fw.write("\n")
    fw.write("[PROM]\n")
    fw.write("knowledge_dir=/opt/knowledge\n")
    fw.write("prev_knowledge_dir=/knowledge\n")
    fw.write("\n")
    fw.write("[CHATGPT]\n")
    fw.write("api_key=?\n")
    fw.write("tokens_limit=0\n")
    fw.write("\n")
    fw.write("[GOOGLE]\n")
    fw.write("search_api_key=?\n")
    fw.write("search_limit=0\n")
    fw.write("\n")
    fw.write("[YANDEX]\n")
    fw.write("spellcheck_api_key=?\n")
    fw.write("spellcheck_limit=0\n")
    fw.write("\n")
    fw.write("[GITHUB]\n")
    fw.write("summary_api_key=?\n")
    fw.write("summary_limit=0\n")
    fw.write("\n")

msg = ""

In [3]:
!pip  install  tiktoken==0.4.0  langchain==0.0.231 openai==0.27.8 faiss-cpu==1.7.4  # gspread oauth2client nltk pydantic==1.10.8

In [5]:
        import configparser
        import pathlib
        import os
        config = configparser.ConfigParser()
        config.read('config.txt')
        knowledge_dir = config["COLAB"]["knowledge_dir"]
        if knowledge_dir is None: knowledge_dir = "./knowledge"
        pathlib.Path(knowledge_dir).mkdir(parents=True, exist_ok=True)
        prev_knowledge_dir = config["COLAB"]["prev_knowledge_dir"]
        if prev_knowledge_dir is None: prev_knowledge_dir = "../knowledge"
        pathlib.Path(prev_knowledge_dir).mkdir(parents=True, exist_ok=True)

        import getpass
        import openai
        import re

        if config["CHATGPT"]["api_key"] == "?":
            openai_key = getpass.getpass("OpenAI API Key:")
        else:
            openai_key = config["CHATGPT"]["api_key"]
        os.environ["OPENAI_API_KEY"] = openai_key
        openai.api_key = openai_key

        from langchain.vectorstores import FAISS
        from langchain.embeddings.openai import OpenAIEmbeddings
        from langchain.text_splitter import RecursiveCharacterTextSplitter
        from langchain.text_splitter import MarkdownHeaderTextSplitter

        def save_file_knowledge(file_path: str, out_path: str) -> str:
            # Чтение текстового файла
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()

            headers_to_split_on = [
                ("#", "Header 1"),
                ("##", "Header 2"),
                ("###", "Header 3"),
                ("####", "Header 4"),
            ]

            markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
            md_header_splits = markdown_splitter.split_text(text)

            # Предполагается, что FAISS и OpenAIEmbeddings были импортированы или определены где-то выше
            vectordateBase = FAISS.from_documents(md_header_splits, OpenAIEmbeddings())
            vectordateBase.save_local(out_path)

            return out_path

        save_file_knowledge(os.path.join(knowledge_dir, "info.md"), os.path.join(knowledge_dir, "info_faiss"))
        save_file_knowledge(os.path.join(knowledge_dir, "howto.md"), os.path.join(knowledge_dir, "howto_faiss"))
        print(msg, " ... OK")


OpenAI API Key:··········
  ... OK
